<a href="https://colab.research.google.com/github/rida-manzoor/DL/blob/main/17_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Agenda of this Notebook
1. Why we can't train our own model?
2. What is transfer Learning?
3. Problem with pre-trained models.
4. Why transfer learning works?

## Why we can't train our own model?
1. Neural Network are data hungary
2. Computation costs
3. Data labelling is an issue.


## Problem with pre-trained models
- **Generalization Limitations:** Pre-trained models may not generalize well to specific or niche domains.

- **Biases:** Models can inherit biases from the data they were trained on, leading to biased outputs.

- **Lack of Domain Knowledge:** Models lack domain-specific knowledge and may not perform optimally in specialized fields.

- **Overfitting to Training Data:** Models might overfit to biases present in the training data, limiting adaptability to new contexts.

- **Computational Resources:** Fine-tuning or adapting large pre-trained models requires substantial computational resources.

# Transfer Learning
Transfer learning is a machine learning technique where a model trained on one task is adapted for a related but different task. Instead of training a model from scratch for a new task, transfer learning leverages knowledge gained from solving one problem to improve performance on another. There are two main types of transfer learning:

1. **Feature-based Transfer Learning:** The pre-trained model's lower layers (features) are used as a generic feature extractor for the new task. The higher layers are then retrained for the specific task. Used for task in which labels are kind of similar to data on which model is already trained.

2. **Fine-tuning:** The entire pre-trained model is further trained on the new task, adjusting weights across all layers to adapt to the specific domain. Used for tasks whose labels are different from data labels on which model is trained.

Transfer learning is particularly useful when the amount of labeled data for the target task is limited, as it enables the model to leverage knowledge gained from larger datasets in related domains.

## Why transfer learning work?
- Early layers only extract primitive features(Edges). And last layer classify image. So Early layer task are generic and will work same for every image. That's why we don't replace them.

##Benefits of Transfer Learning:

**Improved Performance:** Transfer learning can lead to improved performance on the target task, especially when the source and target tasks are related or have some common features.

**Faster Convergence:** By starting with pre-trained weights and learned features, transfer learning can accelerate the training process and reduce the amount of data required for training.

**Effective Use of Limited Data:** Transfer learning is particularly useful when the target dataset is small or limited, as it can leverage knowledge from larger or more diverse datasets.

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

# Load the pre-trained VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add new classification layers on top of the base model
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# Create the transfer learning model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define data generators for training and validation data
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    'train_data_directory',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    'train_data_directory',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples,
    epochs=10
)
